In [20]:
!python -m pip install -U reprozip reprounzip

Requirement already up-to-date: reprozip in /srv/conda/envs/notebook/lib/python3.7/site-packages (1.0.16)
     |████████████████████████████████| 64 kB 924 kB/s eta 0:00:01


In [34]:
%%bash
ls -l
curl -O -L https://github.com/openpreserve/format-corpus/raw/master/variations/variations/application/pdf/lorem-ipsum.pdf
convert lorem-ipsum.pdf lorem-ipsum-%0d.png

total 3416
-rw-r--r-- 1 andy andy   22215 Nov  2 12:38 Getting Started with ReproZip.ipynb
-rw-r--r-- 1 andy andy   21450 Nov  2 12:39 lorem-ipsum.pdf
-rw-r--r-- 1 andy andy 1720320 Nov  2 12:15 ls.rpz
-rw-r--r-- 1 andy andy    1641 Nov  2 12:17 out.dot
-rw-r--r-- 1 andy andy      17 Nov  2 12:02 requirements.txt
-rw-r--r-- 1 andy andy 1720320 Nov  2 12:13 test.rpz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   187  100   187    0     0    413      0 --:--:-- --:--:-- --:--:--   413
100 21450  100 21450    0     0  21710      0 --:--:-- --:--:-- --:--:-- 21710
convert-im6.q16: not authorized `lorem-ipsum.pdf' @ error/constitute.c/ReadImage/412.
convert-im6.q16: no images defined `lorem-ipsum-%0d.png' @ error/convert.c/ConvertImageCommand/3258.


CalledProcessError: Command 'b'ls -l\ncurl -O -L https://github.com/openpreserve/format-corpus/raw/master/variations/variations/application/pdf/lorem-ipsum.pdf\nconvert lorem-ipsum.pdf lorem-ipsum-%0d.png\n'' returned non-zero exit status 1.

In [11]:
!reprozip usage_report --enable

Thank you, usage reports will be sent automatically from now on.


In [14]:
!reprozip trace --overwrite convert

requirements.txt  test.rpz  Untitled.ipynb
Configuration file written in .reprozip-trace/config.yml
Edit that file then run the packer -- use 'reprozip pack -h' for help


In [15]:
!cat .reprozip-trace/config.yml

# ReproZip configuration file
# This file was generated by reprozip 1.0.16 at 2020-11-02T12:14:41+00:00

# You might want to edit this file before running the packer
# See 'reprozip pack -h' for help

# Run info
version: "0.8"
runs:
# Run 0
- architecture: x86_64
  argv:
  - ls
  binary: /bin/ls
  distribution:
  - ubuntu
  - '18.04'
  environ:
    APP_BASE: /srv
    CLICOLOR: '1'
    CONDA_DEFAULT_ENV: notebook
    CONDA_DIR: /srv/conda
    CONDA_EXE: /srv/conda/bin/conda
    CONDA_PREFIX: /srv/conda/envs/notebook
    CONDA_PROMPT_MODIFIER: '(notebook) '
    CONDA_PYTHON_EXE: /srv/conda/bin/python
    CONDA_SHLVL: '1'
    DEBIAN_FRONTEND: noninteractive
    GIT_PAGER: cat
    HOME: /home/andy
    HOSTNAME: 5b9a448eaf37
    JPY_PARENT_PID: '1'
    KERNEL_LAUNCH_TIMEOUT: '40'
    KERNEL_PYTHON_PREFIX: /srv/conda/envs/notebook
    LANG: en_US.UTF-8
    LANGUAGE: en_US.UTF-8
    LC_ALL: en_US.UTF-8
    MPLBACKEND: module://ipykernel.pylab.backend_inline
    NB_PYTHON_PREFIX: /srv/conda/en

In [18]:
!reprozip pack ls.rpz

In [19]:
!file ls.rpz

ls.rpz: POSIX tar archive (GNU)


In [26]:
!reprounzip graph out.dot ls.rpz

In [27]:
!ls -lct

total 3380
-rw-r--r-- 1 andy andy   10553 Nov  2 12:18 'Getting Started with ReproZip.ipynb'
-rw-r--r-- 1 andy andy    1641 Nov  2 12:17  out.dot
-rw-r--r-- 1 andy andy 1720320 Nov  2 12:15  ls.rpz
-rw-r--r-- 1 andy andy 1720320 Nov  2 12:13  test.rpz
-rw-r--r-- 1 andy andy      17 Nov  2 12:09  requirements.txt


In [28]:
!file out.dot

out.dot: ASCII text


In [29]:
!cat out.dot

digraph G {
    rankdir=LR;

    /* programs */
    node [shape=box fontcolor=white fillcolor=black style="filled,rounded"];
    subgraph cluster_run0 {
        label="run0";
        prog0 [label="/bin/ls (1)"];
    }

    node [shape=ellipse fontcolor="#131C39" fillcolor="#C9D2ED"];

    /* system packages */
    subgraph cluster_pkg0 {
        label="coreutils 8.28-1ubuntu1";
        "/bin/ls";
    }
    subgraph cluster_pkg1 {
        label="libc6 2.27-3ubuntu1.2";
        "/lib/x86_64-linux-gnu/ld-2.27.so";
        "/lib/x86_64-linux-gnu/libc.so.6";
        "/lib/x86_64-linux-gnu/libdl.so.2";
        "/lib/x86_64-linux-gnu/libpthread.so.0";
    }
    subgraph cluster_pkg2 {
        label="libpcre3 2:8.39-9";
        "/lib/x86_64-linux-gnu/libpcre.so.3";
    }
    subgraph cluster_pkg3 {
        label="libselinux1 2.7-2build2";
        "/lib/x86_64-linux-gnu/libselinux.so.1";
    }

    /* other files */
    "/etc/ld.so.cache";
    "/proc/filesystems";
    "/usr/lib/locale/locale-ar